In [0]:
import requests
import uuid
from datetime import datetime
from pyspark.sql.functions import col, lit, to_timestamp,udf, explode,from_unixtime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType, ArrayType
from delta.tables import *

In [0]:
spark.catalog.clearCache()

In [0]:
%sql
DROP TABLE IF EXISTS data_log_table

In [0]:
dbutils.fs.rm("/user/hive/warehouse/data_log_table", recurse=True)


In [0]:
%sql
CREATE OR REPLACE TABLE data_log_table (
  id STRING,
  load_type STRING,
  table_name STRING,
  process_start_time TIMESTAMP,
  process_end_time TIMESTAMP,
  status STRING,
  comments STRING,
  start_date_time TIMESTAMP,
  end_date_time TIMESTAMP,
  created_on TIMESTAMP,
  created_by STRING
)
USING DELTA;


In [0]:
def logs(func):
    def wrapper(*args, **kwargs):
        id = str(uuid.uuid4())
        load_type = args[0]
        table_name = args[1]
        process_start_dt = datetime.now()
        name = 'Eva Shrestha'

        spark.sql(f"INSERT INTO data_log_table (id, load_type, table_name, process_start_time, status, created_on, created_by) \
            VALUES ('{id}', '{load_type}', '{table_name}', '{process_start_dt}', 'EXTRACTING', '{process_start_dt}', '{name}')")


        df, start, end = func(*args[2:], **kwargs)

        df = df.withColumn('load_run_id', lit(id))
        df = df.withColumn('created_on', lit(process_start_dt).cast(TimestampType()))
        df = df.withColumn('created_by', lit(name))

        df.write.format('delta').mode('append').saveAsTable(table_name)

        process_end_dt = datetime.now()
        spark.sql(f"UPDATE data_log_table SET process_end_time = '{process_end_dt}', status='COMPLETED', start_date_time='{start}', end_date_time='{end}' WHERE id='{id}'")

        return df

    return wrapper